In [1]:
import pandas as pd

# Data Staging: 
## Extract, Transform and Load (ETL)

## Conceptual model:

<img src = "4142DIAGRAM.png" style ="width: 800px">

# Extract

In [2]:
df= pd.read_csv('./healthcare-dataset-stroke-data.csv')

## Updating column names to to ensure uniformity 

In [3]:
df = df.rename({'Residence_type' : 'residence_type'},axis = 'columns')
df = df.replace( {'formerly smoked':'Formerly smoked', 'never smoked': 'Never smoked', 'smokes':'Smokes', 'children': 'Children'})

Dropping missing values

In [4]:
df = df.dropna()

Ensuring missing values have been dropped

In [5]:
df.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

## Binning Data 

Starting with binning age group

In [6]:
df['age_group'] = pd.cut(df['age'], bins=[0,10,20,30,40,50,60,70,80,90,100], right=False, labels= ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90+'])

Binning BMI

In [7]:
df['bmi_category'] = pd.cut(df['bmi'],bins=[0,18.5,25,30,40,100], right=False, labels= ['Underweight','Normal weight', 'Obese class I', 'Obese class II', 'Morbidly obese'])

Binning glucose

In [8]:
df['glucose_range'] = pd.cut(df['avg_glucose_level'],bins=[0,200,500], right=False, labels= ['Non-diabetic','Diabetic'])

Binning smoking status

In [9]:
#df['ever_smoked'] = df['smoking_status']

df['ever_smoked'] = df['smoking_status'].replace( {'Formerly smoked':'Yes', 'Never smoked': 'No', 'Smokes':'Yes', 'Children': 'No'})

Binning number of risk factors each person has (glucose_range, smoked, bmi, hypertension, heart disease)

In [10]:
def riskChecker(x):
    counter = 0 
    tmpHypertension = x.hypertension
    tmpHeartDisease = x.heart_disease
    tmpBMI = x.bmi_category
    tmpGlucose = x.glucose_range
    tmpSmoked = x.ever_smoked
    if tmpHypertension == 1:
        counter += 1
    if tmpHeartDisease == 1: 
        counter += 1
    if tmpBMI == 'Obese class I' or tmpBMI == 'Obese class II' or tmpBMI ==  'Morbidly obese': 
        counter += 1
    if tmpGlucose == 'Diabetic':
        counter += 1 
    if tmpSmoked == x.ever_smoked == 'Yes':
        counter += 1
    
    return counter


In [11]:
df['risk_factor_count'] = df.apply(riskChecker, axis = 1)

## Generating Surrogate Keys

In [12]:
df['surr_key'] = range(1,len(df)+1)

Dropping ID column

In [13]:
df = df.drop('id', axis=1)

In [14]:
df

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_group,bmi_category,glucose_range,ever_smoked,risk_factor_count,surr_key
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,Formerly smoked,1,60-69,Obese class II,Diabetic,Yes,4,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,Never smoked,1,80-89,Obese class II,Non-diabetic,No,2,2
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,Smokes,1,40-49,Obese class II,Non-diabetic,Yes,2,3
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,Never smoked,1,70-79,Normal weight,Non-diabetic,No,1,4
5,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,Formerly smoked,1,80-89,Obese class I,Non-diabetic,Yes,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,Female,13.0,0,0,No,Children,Rural,103.08,18.6,Unknown,0,10-19,Normal weight,Non-diabetic,Unknown,0,4905
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,Never smoked,0,80-89,Morbidly obese,Non-diabetic,No,1,4906
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,Never smoked,0,30-39,Obese class II,Non-diabetic,No,1,4907
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,Formerly smoked,0,50-59,Obese class I,Non-diabetic,Yes,2,4908


# Creating and Inputting into DB

In [22]:

fact_table_dataframe = pd.DataFrame(df,
                  columns = ['total_strokes','patient_count', 'number_smokers', 'demo_key','health_key','personal_key'])
personal_attribute_table = pd.DataFrame(df,
                  columns = ['surr_key', 'gender','age'])
demographic_info_table = pd.DataFrame(df,
                  columns = ['surr_key','ever_married', 'work_type','residence_type'])
health_attribute_table = pd.DataFrame(df,
                  columns = ['surr_key','hypertension', 'heart_disease','smoking_status',
                             'avg_glucose_level','bmi'])

print(fact_table_dataframe)
                             

      total_strokes  patient_count  number_smokers  demo_key  health_key  \
0               NaN            NaN             NaN       NaN         NaN   
2               NaN            NaN             NaN       NaN         NaN   
3               NaN            NaN             NaN       NaN         NaN   
4               NaN            NaN             NaN       NaN         NaN   
5               NaN            NaN             NaN       NaN         NaN   
...             ...            ...             ...       ...         ...   
5104            NaN            NaN             NaN       NaN         NaN   
5106            NaN            NaN             NaN       NaN         NaN   
5107            NaN            NaN             NaN       NaN         NaN   
5108            NaN            NaN             NaN       NaN         NaN   
5109            NaN            NaN             NaN       NaN         NaN   

      personal_key  
0              NaN  
2              NaN  
3              NaN  
4  

In [ ]:
def countSmokers(x):
    counter = 0
    tmpSmoker = x.ever_smoked
    if tmpSmoker == 'Yes':
        counter += 1
    return counter
    

# Insert DataFrame recrds one by one.
total_stroke = df['stroke'].sum()
patient_count = len(df)
smoker_count = countSmokers(df)


In [16]:
personal_attribute_table = personal_attribute_table.rename({'surr_key' : 'personal_key'},axis = 'columns')
demographic_info_table = demographic_info_table.rename({'surr_key' : 'demo_key'},axis = 'columns')
health_attribute_table = health_attribute_table.rename({'surr_key' : 'health_key'},axis = 'columns')

In [17]:
fact_table_cols = ",".join([str(i) for i in fact_table_dataframe.columns.tolist()])
personal_attribute_cols = ",".join([str(i) for i in personal_attribute_table.columns.tolist()])
demographic_info_cols = ",".join([str(i) for i in demographic_info_table.columns.tolist()])
health_attribute_cols = ",".join([str(i) for i in health_attribute_table.columns.tolist()])

In [19]:
from configparser import ConfigParser
import psycopg2
from typing import Dict


def load_connection_info(
    ini_filename: str
) -> Dict[str, str]:
    parser = ConfigParser()
    parser.read(ini_filename)
    # Create a dictionary of the variables stored under the "postgresql" section of the .ini
    conn_info = {param[0]: param[1] for param in parser.items("postgresql")}
    return conn_info


def create_db(
    conn_info: Dict[str, str],
) -> None:
    # Connect just to PostgreSQL with the user loaded from the .ini file
    psql_connection_string = f"user={conn_info['user']} password={conn_info['password']}"
    conn = psycopg2.connect(psql_connection_string)
    cur = conn.cursor()

    # "CREATE DATABASE" requires automatic commits
    conn.autocommit = True
    sql_query = f"CREATE DATABASE {conn_info['database']}"

    try:
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        cur.close()
    else:
        # Revert autocommit settings
        conn.autocommit = False


def create_table(
    sql_query: str, 
    conn: psycopg2.extensions.connection, 
    cur: psycopg2.extensions.cursor
) -> None:
    try:
        # Execute the table creation query
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        conn.rollback()
        cur.close()
    else:
        # To take effect, changes need be committed to the database
        conn.commit()

if __name__ == "__main__":
    # host, database, user, password
    conn_info = load_connection_info("db.ini")

    # Create the desired database
    create_db(conn_info)

    # Connect to the database created
    connection = psycopg2.connect(**conn_info)
    cursor = connection.cursor()

    # Create the "demographic_info" table
    demographic_info = """
        CREATE TABLE demographic_info (
            demo_key INTEGER PRIMARY KEY,
            ever_married TEXT,
            work_type TEXT,
            residence_type TEXT
        )
    """
    create_table(demographic_info, connection, cursor)



    # Create the "health_attribute" table
    health_attribute = """
        CREATE TABLE health_attribute (
            health_key INTEGER PRIMARY KEY,
            hypertension INTEGER,
            heart_disease INTEGER,
            smoking_status TEXT,
            avg_glucose_level DOUBLE PRECISION,
            bmi DOUBLE PRECISION
        )
    """
    create_table(health_attribute, connection, cursor)

    # Create the "personal_attributes" table
    personal_attributes  = """
        CREATE TABLE personal_attributes (
            personal_key INTEGER PRIMARY KEY,
            gender TEXT,
            age DOUBLE PRECISION
        )
    """
    create_table(personal_attributes, connection, cursor)

    # Create the "fact_table" table
    fact_table = """
        CREATE TABLE fact_table (
            total_strokes INTEGER,
            paitent_count INTEGER,
            Number_smokers INTEGER,
            demo_key INTEGER REFERENCES demographic_info(demo_key),
            health_key INTEGER REFERENCES health_attribute(health_key),
            personal_key INTEGER REFERENCES personal_attributes(personal_key)
        )
    """
    create_table(fact_table, connection, cursor)


    
    
    
    

    # Adding fact table
    for i,row in fact_table_dataframe.iterrows():
        fact_table_sql = "INSERT INTO fact_table ( total_strokes, patient_count, number_smokers, demo_key, health_key, personal_key) VALUES (%s, %s, %s,%s, %s, %s)"
        cursor.execute(fact_table_sql, tuple(row))
        connection.commit()
    

    # Adding personal attribute table
    for i,row in personal_attribute_table.iterrows():
        personal_attr_sql = "INSERT INTO personal_attributes (personal_key, gender, age) VALUES (%s, %s, %s)"
        cursor.execute(personal_attr_sql, tuple(row))
        connection.commit()
        
    # Adding demographic info table
    for i,row in demographic_info_table.iterrows():
        demographic_info_sql = "INSERT INTO demographic_info (demo_key,ever_married,work_type,residence_type) VALUES (%s, %s, %s,%s)"
        cursor.execute(demographic_info_sql, tuple(row))
        connection.commit()

    # Adding health attribute table
    for i,row in health_attribute_table.iterrows():
        health_attribute_sql = "INSERT INTO health_attribute (health_key,hypertension,heart_disease,smoking_status,avg_glucose_level,bmi) VALUES (%s, %s, %s,%s,%s,%s)"
        cursor.execute(health_attribute_sql, tuple(row))
        connection.commit()
        
    connection.close()
    cursor.close()


DuplicateDatabase: database "stroke_db" already exists

Query: b'CREATE DATABASE stroke_db'
DuplicateTable: relation "demographic_info" already exists

Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_married TEXT,\n            work_type TEXT,\n            residence_type TEXT\n        )\n    '
InterfaceError: cursor already closed
Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_married TEXT,\n            work_type TEXT,\n            residence_type TEXT\n        )\n    '
InterfaceError: cursor already closed
Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_married TEXT,\n            work_type TEXT,\n            residence_type TEXT\n        )\n    '
InterfaceError: cursor already closed
Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_marr

InterfaceError: cursor already closed